In [10]:
import pandas as pd
from tqdm import tqdm

In [11]:
txt_file = "./data/Hi-small_Patterns.txt"
csv_file = "./data/HI-Small_Trans.csv"

In [12]:
df = pd.read_csv(csv_file)

In [13]:
txt_lines = open(txt_file, "r").read()

In [14]:
laundering_patterns = txt_lines.split("BEGIN")

In [8]:
df['laundering_pattern'] = ['' for _ in range(len(df))]

In [16]:
trans_dict = {}

for idx, row in tqdm(df.iterrows()):
    from_account = row["Account"]
    to_account = row["Account.1"]

    trans_dict[f"{from_account}_{to_account}"] = idx


5078345it [01:23, 60895.99it/s]


In [67]:
for pattern in tqdm(laundering_patterns):
    transactions = pattern.split('\n')
    transactions_pattern = transactions[0].split(' ')[-1]
    for trans in transactions[1:]:
        if trans[:3] == 'END':
            break
        from_account = trans.split(',')[2]
        to_account = trans.split(',')[4]
        key = f"{from_account}_{to_account}"
        df.loc[trans_dict[key], 'laundering_pattern'] = transactions_pattern     

100%|███████████████████████████████████████| 371/371 [00:00<00:00, 1659.41it/s]
